# Short enwiki batch testing

Test that the training code, batching mode, works with

**1B5 model / L24-D2048 model with**
- Layer count: 24
- Embed size: 2048

## The following are the performance numbers on a single AWS A10G

| Batch Size | Peak Tokens / Sec | Peak VRAM  | Time taken (include setup/validation) |
|------------|-------------------|------------|---------------------------------------|
| 8          | 53.71 kt/sec      | ~ 20.09 GB | 71.45 sec                             |
| 4          | 44.17 kt/sec      | ~ 13.24 GB | 83.63 sec                             |
| 2          | 32.51 kt/sec      | ~ 7.74 GB  | 108.472 sec                           |
| 1          | 21.73 kt/sec      | ~ 4.91 GB  | 153.902 sec                           |

The general advice is increase your microbatch_size size until your vram maxes out. Then increase target_batch_size to match, or scale in multiples of GPU count & microbatch_size

## Preparing the init model and test dataset

In [2]:
GPU_DEVICES="auto"
ENABLE_WANDB=False
WANDB_PREFIX="infctx-v5-validation"
DEEPSPEED_STRAT="deepspeed_stage_2_offload"

print("ENABLE_WANDB:", ENABLE_WANDB)
print("GPU_DEVICES:", GPU_DEVICES)

if ENABLE_WANDB:
    WANDB_MODE="online"
else:
    WANDB_MODE="disabled"

# Computing the notebook, and various paths
import os
NOTEBOOK_DIR=os.path.dirname(os.path.abspath("__file__"))
PROJECT_DIR=os.path.abspath(os.path.join(NOTEBOOK_DIR, "../../"))
TRAINER_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v5/"))

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("TRAINER_DIR:", TRAINER_DIR)
print("PROJECT_DIR:", PROJECT_DIR)

ENABLE_WANDB: False
GPU_DEVICES: auto
NOTEBOOK_DIR: /home/picocreator/rwkv-proj/RWKV-infctx-trainer/notebook/trainer-v5-validation
TRAINER_DIR: /home/picocreator/rwkv-proj/RWKV-infctx-trainer/RWKV-v5
PROJECT_DIR: /home/picocreator/rwkv-proj/RWKV-infctx-trainer


In [2]:
# First lets setup the various directories
!mkdir -p "{PROJECT_DIR}/model/"
!mkdir -p "{PROJECT_DIR}/datapath/"
!mkdir -p "{PROJECT_DIR}/checkpoint/"

In [3]:
# Lets initialized the L24-D2048 model with the init_model.py code
!cd "{TRAINER_DIR}" && python3 init_model.py \
    --n_layer 24 --n_embd 2048 \
    --vocab_size world \
    --skip-if-exists --safe-init \
    ../model/L24-D2048-world-init.pth

[2023-11-19 12:25:47,003] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.1.0'
---- Initializing model ----
No of layers: 24
Embedding size: 2048
Output model path: ../model/L24-D2048-world-init.pth
Vocab size: 65536
Emb scale: 0.0001
Note: this process takes a significant time (and ram) for large models
---- ----- ----
---
[RWKV.TimeMix] Compiling CUDA kernel with HEAD_SIZE=64
Using /home/picocreator/.cache/torch_extensions/py311_cu121 as PyTorch extensions root...
Creating extension directory /home/picocreator/.cache/torch_extensions/py311_cu121/wkv5...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/picocreator/.cache/torch_extensions/py311_cu121/wkv5/build.ninja...
Building extension module wkv5...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
[1/3] c++ -MMD -MF wkv5_op.o.d -DTORCH_EXT

In [4]:
# Preload the dataset
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/config/enwiki_10k-world-4096.yaml"

Extracting data files: 100%|████████████████████| 1/1 [00:00<00:00, 1316.89it/s]
Setting num_proc from 16 back to 1 for the train split to disable multiprocessing as it only contains one shard.
Map (num_proc=16): 100%|███████████| 1339/1339 [00:00<00:00, 2247.10 examples/s]
Saving the dataset (1/1 shards): 100%|█| 763/763 [00:00<00:00, 27513.45 examples
Saving the dataset (1/1 shards): 100%|███| 8/8 [00:00<00:00, 4063.26 examples/s]


In [3]:
# Short training process - for quick testing / debugging
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="disabled" && \
    python3 lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/config/enwiki_10k-world-4096.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} (train-ctx=1024, data-ctx=4096, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}" \
        --trainer.fast_dev_run=2 \
        --model.ctx_len=256 \
        --trainer.microbatch_size=1 \
        --model.load_model="../model/L24-D2048-world-init.pth"

[2023-11-19 13:29:31,402] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.1.0'
/home/picocreator/anaconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/pytorch/cli.py:518: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/home/picocreator/rwkv-proj/RWKV-infctx-trainer/notebook/trainer-v5-validation/config/enwiki_10k-world-4096.yaml', '--trainer.logger.init_args.name=infctx-v5-validation (train-ctx=1024, data-ctx=4096, deepspeed_stage_2)', '--trainer.strategy=deepspeed_stage_2', '--trainer.devices=auto', '--trainer.fast_dev_run=2', '--model.ctx_len=256', '--trainer.microbatch_size=1', '--model.load_model=../model/L24-D2048-world-init.pth'], args=['fit', '-c', '/home/picocreator/rwkv-proj/R

In [15]:
# Empty out the checkpoint
!cd "{PROJECT_DIR}" && rm -rf "./checkpoint/infctx-v5-unit-test-baseline-4096/"

# Microbatch 1 training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python3 lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/config/enwiki_10k-world-4096.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - microbatch 1 (train-ctx=4096, data-ctx=4096, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}" \
        --trainer.microbatch_size=1 \
        --model.load_model="../model/L24-D2048-world-init.pth"
        

[2023-11-19 02:13:59,508] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.1.0'
/home/ubuntu/anaconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/pytorch/cli.py:518: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/home/ubuntu/rwkv-proj/RWKV-infctx-trainer/notebook/trainer-v5-validation/config/enwiki_10k-world-4096.yaml', '--trainer.logger.init_args.name=infctx-v5-validation - microbatch 1 (train-ctx=4096, data-ctx=4096, deepspeed_stage_1)', '--trainer.strategy=deepspeed_stage_1', '--trainer.devices=auto', '--trainer.microbatch_size=1', '--model.load_model=../model/L6-D512-world-init.pth'], args=['fit', '-c', '/home/ubuntu/rwkv-proj/RWKV-infctx-trainer/notebook/trainer-v5-validation/con

In [16]:
# Empty out the checkpoint
!cd "{PROJECT_DIR}" && rm -rf "./checkpoint/infctx-v5-unit-test-baseline-4096/"

# Microbatch 1 training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python3 lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/config/enwiki_10k-world-4096.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - microbatch 2 (train-ctx=4096, data-ctx=4096, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}" \
        --trainer.microbatch_size=2 \
        --model.load_model="../model/L24-D2048-world-init.pth"

[2023-11-19 02:16:53,648] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.1.0'
/home/ubuntu/anaconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/pytorch/cli.py:518: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/home/ubuntu/rwkv-proj/RWKV-infctx-trainer/notebook/trainer-v5-validation/config/enwiki_10k-world-4096.yaml', '--trainer.logger.init_args.name=infctx-v5-validation - microbatch 2 (train-ctx=4096, data-ctx=4096, deepspeed_stage_1)', '--trainer.strategy=deepspeed_stage_1', '--trainer.devices=auto', '--trainer.microbatch_size=2', '--model.load_model=../model/L6-D512-world-init.pth'], args=['fit', '-c', '/home/ubuntu/rwkv-proj/RWKV-infctx-trainer/notebook/trainer-v5-validation/con

In [17]:
# Empty out the checkpoint
!cd "{PROJECT_DIR}" && rm -rf "./checkpoint/infctx-v5-unit-test-baseline-4096/"

# Microbatch 1 training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python3 lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/config/enwiki_10k-world-4096.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - microbatch 3 (train-ctx=4096, data-ctx=4096, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}" \
        --trainer.microbatch_size=3 \
        --model.load_model="../model/L24-D2048-world-init.pth"

[2023-11-19 02:19:00,550] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.1.0'
/home/ubuntu/anaconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/pytorch/cli.py:518: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/home/ubuntu/rwkv-proj/RWKV-infctx-trainer/notebook/trainer-v5-validation/config/enwiki_10k-world-4096.yaml', '--trainer.logger.init_args.name=infctx-v5-validation - microbatch 4 (train-ctx=4096, data-ctx=4096, deepspeed_stage_1)', '--trainer.strategy=deepspeed_stage_1', '--trainer.devices=auto', '--trainer.microbatch_size=4', '--model.load_model=../model/L6-D512-world-init.pth'], args=['fit', '-c', '/home/ubuntu/rwkv-proj/RWKV-infctx-trainer/notebook/trainer-v5-validation/con

In [18]:
# Empty out the checkpoint
!cd "{PROJECT_DIR}" && rm -rf "./checkpoint/infctx-v5-unit-test-baseline-4096/"

# Microbatch 1 training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python3 lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/config/enwiki_10k-world-4096.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - microbatch 8 (train-ctx=4096, data-ctx=4096, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}" \
        --trainer.microbatch_size=4 \
        --model.load_model="../model/L24-D2048-world-init.pth"

[2023-11-19 02:20:44,542] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.1.0'
/home/ubuntu/anaconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/pytorch/cli.py:518: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/home/ubuntu/rwkv-proj/RWKV-infctx-trainer/notebook/trainer-v5-validation/config/enwiki_10k-world-4096.yaml', '--trainer.logger.init_args.name=infctx-v5-validation - microbatch 8 (train-ctx=4096, data-ctx=4096, deepspeed_stage_1)', '--trainer.strategy=deepspeed_stage_1', '--trainer.devices=auto', '--trainer.microbatch_size=8', '--model.load_model=../model/L6-D512-world-init.pth'], args=['fit', '-c', '/home/ubuntu/rwkv-proj/RWKV-infctx-trainer/notebook/trainer-v5-validation/con